In [1]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

sys.path.append("C:\\Users\\zuk-8\\Programs\\workspace\\fantacalcio\\src")
# sys.path.append("C:\\Users\\zuk-8\\Programs\\workspace\\fantacalcio\\src\\fantacalcio")
# sys.path.append("C:\\Users\\zuk-8\\Programs\\workspace\\fantacalcio\\src\\fantacalcio\\config_data")
# sys.path.append("C:\\Users\\zuk-8\\Programs\\workspace\\fantacalcio\\src\\fantacalcio\\data")

from fantacalcio.config_data.config_data import FOLDER_FANTA_INPUT, FANTA_GRADES_FN, FANTA_QUOTES_FN, FANTA_REMAP_FN
from fantacalcio.data.general import canceled_matches_grades_to_nan, remap_players
from fantacalcio.data.player import Player

In [2]:
# INPUTS
average_grades_by_role = pd.Series({'P':4.5, 'D':5.5, 'C':5.75, 'T':6.2, 'A':6.4})

In [3]:
# DATA
GRADES = canceled_matches_grades_to_nan(pd.read_csv(os.path.join(FOLDER_FANTA_INPUT[os.environ['USERNAME']], FANTA_GRADES_FN)))
REMAP_PLAYERS = pd.read_csv(os.path.join(FOLDER_FANTA_INPUT[os.environ['USERNAME']], FANTA_REMAP_FN)).\
    set_index('Original')['Final'].to_dict()
GRADES = remap_players(GRADES, REMAP_PLAYERS)

# Penalties:
# 'R'  is the number of scored penalties
# 'RS' is the number of missed penalties, only for players and not for gks
penalty_y = GRADES.ix[~GRADES['R'].map(pd.np.isnan)]
penalty_n = GRADES.ix[~GRADES['RS'].map(pd.np.isnan)]
GRADES = GRADES.set_value(penalty_y.index, col=['FantaVoto'], value=penalty_y['FantaVoto']-penalty_y['R']*3)
GRADES = GRADES.set_value(penalty_n.index, col=['FantaVoto'], value=penalty_y['FantaVoto']+penalty_y['RS']*3)


C:\Users\zuk-8\Programs\Anaconda3\envs\standard\lib\site-packages\ipykernel\__main__.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
C:\Users\zuk-8\Programs\Anaconda3\envs\standard\lib\site-packages\ipykernel\ipkernel.py:196: PerformanceWarning: indexing past lexsort depth may impact performance.
  res = shell.run_cell(code, store_history=store_history, silent=silent)


In [4]:
# SCORES by season
matches_bs = GRADES.reset_index().groupby(['Season', 'Player'])['FantaVoto'].count().to_frame('Matches')
no_matches_bs = (38 - matches_bs['Matches']).to_frame('No_Matches')
fv_grades_bs = GRADES.reset_index().groupby(['Season', 'Player'])['FantaVoto'].mean().to_frame('Mean_FV')
v_grades_bs = GRADES.reset_index().groupby(['Season', 'Player'])['Voto'].mean().to_frame('Mean_V')
fv_grades_tot_bs = GRADES.reset_index().groupby(['Season', 'Player'])['FantaVoto'].sum().to_frame('Total_FV')
players = GRADES.reset_index()['Player'].unique()
roles_bs = GRADES.reset_index().sort_values(['Season', 'Giornata']).groupby(['Season', 'Player'])['Role'].last().to_frame('Role')
roles = roles_bs.reset_index().groupby('Player')['Role'].last().to_frame('Role')
# roles['Role'] = roles['Role'].map(lambda x:'A' if x=='T' else x)

nm_bs = no_matches_bs.reset_index()
nm_bs['avg'] = nm_bs.Player.map(roles['Role']).map(average_grades_by_role)
nm_bs = nm_bs.set_index(['Season', 'Player'])
average_score_bs = ((nm_bs['No_Matches'] * nm_bs['avg'] + fv_grades_tot_bs['Total_FV'])/38.).to_frame('average_score')

penalty_by_team_all_bs = GRADES.reset_index().query("Role != 'P'").groupby(['Season', 'Team'])[['R', 'RS']].sum().fillna(0)
penalty_by_team_bs = (penalty_by_team_all_bs['R']+penalty_by_team_all_bs['RS']).fillna(0).to_frame('team_penalties')
team_players_bs = GRADES.reset_index().sort_values(['Season', 'Giornata']).groupby(['Season', 'Player']).last()['Team']
penalty_by_team_for_players_bs = team_players_bs.reset_index()\
    .merge(penalty_by_team_bs.reset_index(), left_on=['Season', 'Team'], right_on=['Season', 'Team'])\
    .set_index(['Season', 'Player'])['team_penalties']
penalties_by_player_bs = GRADES[['R', 'RS']].fillna(0).reset_index().groupby(['Season', 'Player'])[['R', 'RS']].sum()
total_penalties_bs = penalties_by_player_bs.sum(1).to_frame('Penalties_Kicked')
percentage_penalties_scored_bs = (penalties_by_player_bs['R']/total_penalties_bs['Penalties_Kicked']).\
    to_frame('Percentage_Penalties_Scored')
percentage_team_penalties_kicked_bs = (total_penalties_bs['Penalties_Kicked'] / penalty_by_team_for_players_bs).\
    to_frame('Percentage_Team_Penalties_Kicked')

scores_bs = matches_bs.join(fv_grades_bs, how='outer').join(average_score_bs, how='outer').join(v_grades_bs, how='outer')\
    .join(total_penalties_bs, how='outer').join(percentage_penalties_scored_bs, how='outer')\
    .join(percentage_team_penalties_kicked_bs, how='outer')\
    .reset_index().merge(roles.reset_index(), how='outer', left_on='Player', right_on='Player')\
    .sort_values(['Season', 'Role', 'average_score']).set_index(['Season', 'Role', 'Player'])
scores_bs.to_clipboard()


In [75]:
def get_standard_stats(name):
    all_players = [str(p) for p in GRADES.reset_index()['Player'].unique().tolist()]
    all_players.sort()
    players = [player for player in all_players if str(player).find(name)>=0]
    if len(players)==1:
        full_name = players[0]
        pl = Player(full_name)
        pl.subplot(['Voto', 'FantaVoto'], figsize=(15,3))
        ma = pl.all_grades.reset_index().groupby('Season')['FantaVoto'].count().to_frame('Matches')
        gr = pl.all_grades.reset_index().groupby('Season').mean()[['Voto', 'FantaVoto']]
        print(', '.join([pl.name, pl.role, pl.current_team, str(int(pl.player_age)), pl.player_nation]))
#         print(pl.quotes)
        print(ma.join(gr))
    else:
        print(players)

In [102]:
name = 'Cutrone'
get_standard_stats(name)
# pl = Player(name)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'